# Rapport du projet Navigation - Cours 2 "Basé sur la fonction de valeur" - Programme nanodegré "Apprentissage profond par renforcement" - Udacity

# L'objectif

Dans l'environnement Unity "Banana" fourni, il s'agit d'entrainer un agent à récolter les bananes jaunes, tout en évitant les bananes bleues, durant un épisode de 300 pas maximum (imposé par l'environnement), afin d'atteindre un score supérieur ou égal à 13. L'objectif consiste à obtenir une moyenne de score glissante sur 100 épisodes supérieure ou égale à 13. Un objectif optionnel consiste à entrainer l'agent à atteindre cet objectif en moins de 1800 épisodes.

Pour cela, l'agent reposera sur une méthode d'apprentissage par renforcement, Deep Q-Network (DQN), basée sur la fonction de valeur qui est estimée par un réseau de neurones.

## La méthode

Cet algorithme DQN est implémenté en python avec la librairie [PyTorch](https://pytorch.org/). Il s'agit d'un algorithme d'apprentissage par renforcement qui utilise un réseau de neurones pour approximer la fonction de valeur d'action. L'agent apprend à partir des récompenses qu'il reçoit en interagissant avec l'environnement, en ajustant ses actions pour maximiser la récompense cumulative à long terme.

Le réseau de neurones utilisé comme estimateur introduit une instabilité dans l'apprentissage. Pour remédier à cela, nous utilisons un replay memory pour stocker les expériences passées et un réseau cible pour réduire cette instabilité. Le réseau cible est mis à jour moins fréquemment que le réseau principal, ce qui permet de réduire la variance des mises à jour.

Comme l'utilisation de l'algorithme DQN nécessite un grand nombre d'itérations pour converger vers une politique optimale, il est essentiel de mettre en place des mécanismes d'exploration et d'exploitation. L'agent doit explorer différentes actions pour découvrir celles qui mènent à des récompenses élevées, tout en exploitant les connaissances acquises pour maximiser la récompense à chaque étape.

La boucle d'apprentissage repose sur une mise à jour régulière des poids du réseau de neurones, en utilisant la fonction de perte entre la valeur d'action estimée et la valeur d'action cible. La fonction de perte utilisée est l'erreur quadratique moyenne (MSE), qui mesure la différence entre les valeurs prédites par le réseau et les valeurs cibles.

Entre chaque mise à jour, l'agent interagit un nombre déterminé de fois avec l'environnement en choisissant des actions basées sur une stratégie $\epsilon$-greedy. Cela signifie qu'il choisit une action aléatoire avec une probabilité $\epsilon$, et la meilleure action connue avec une probabilité $1 - \epsilon$. Cette stratégie permet à l'agent d'explorer de nouvelles actions tout en exploitant les connaissances acquises.

Ces interactions sont stockées dans le replay memory, qui est un tampon circulaire de taille fixe. Lors de chaque mise à jour, un échantillon aléatoire d'expériences est extrait du replay memory pour entraîner le réseau de neurones sur un batch d'interactions. Cela permet de briser la corrélation entre les expériences successives et d'améliorer la stabilité de l'apprentissage.

Le réseau de neurones utilisé dans l'algorithme DQN est un réseau linéaire qui prend pour entrée l'état de l'environnement et produit en sortie les valeurs d'action pour chaque action possible. La taille de l'entrée du réseau correspond à la taille de l'espace d'état de l'environnement, et la taille de la sortie correspond au nombre d'actions possibles.

In [1]:
from unityagents import UnityEnvironment
env = UnityEnvironment(file_name="Banana.app")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

Mono path[0] = '/Users/me/Dropbox (Compte personnel)/Perso NG/Cours et Mooc/Udacity/Deep Reinforcement learning/Cours 2 - Value-Based Methods/Udacity Course 2 Project/Udacity Course 2 Project - Source/Banana.app/Contents/Resources/Data/Managed'
Mono config path = '/Users/me/Dropbox (Compte personnel)/Perso NG/Cours et Mooc/Udacity/Deep Reinforcement learning/Cours 2 - Value-Based Methods/Udacity Course 2 Project/Udacity Course 2 Project - Source/Banana.app/Contents/MonoBleedingEdge/etc'


INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Voici un aperçu de l'architecture du réseau de neurones utilisé dans l'algorithme DQN, par rapport à l'environnement "Banana" :

In [2]:
from models.linear_v1 import QNetworkLinear

model = QNetworkLinear(
    state_size=brain.vector_observation_space_size,
    action_size=brain.vector_action_space_size,
)
print(model)

QNetworkLinear(
  (fc1): Linear(in_features=37, out_features=64, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=64, out_features=64, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=64, out_features=4, bias=True)
)


## Les résultats

### L'entrainement

Nous obtenons les résultats suivants en exécutant la partie entrainement du carnet `Training DQN Agent.ipynb` sur un MacbookAir M2 :

<img src="./images/model_weights_275_solved training stats.png" alt="Stats from training with Training DQN Agent.ipynb notebook" width="600"/>

- L'apprentissage de l'agent permet de résoudre l'environnement en 521 épisodes (bien en dessous des 1800 épisodes indiqués pour le défi), avec une contrainte de nombre de pas de temps par épisode de 250 (au lieu des 300 imposés par l'environnement).
- On constate que l'agent résoud de plus en plus rapidement l'environnement au fur et à mesure de l'entrainement, comme l'indique la courbe de moyenne des pas de temps pour réussir à résoudre l'environnement (c'est à dire atteindre le score de 13).

Les poids du modèle associé à l'agent sont sauvegardés dans le fichier `model_weights_275_solved.pth` et peuvent être utilisés pour tester l'agent dans l'environnement.

### L'évaluation

Nous obtenons les résultats suivants en exécutant la partie évaluation du carnet `Training DQN Agent.ipynb`:

<img src="./images/model_weights_275_solved evaluation stats.png" alt="Stats from training with Training DQN Agent.ipynb notebook" width="600"/>

Nous constatons que les objectifs fixés ont été atteints, même avec des contraintes plus fortes sur le nombre d'épisodes pour résoudre le problème, le nombre maximum de pas de temps disponibles par épisode, ainsi qu'avec la stabilité de la moyenne glissante sur 100 épisodes. Nous remarquons aussi que bien qu'après 100 épisodes la moyenne glissante est stable. En fin, seul un épisode réussi ne l'est pas en respectant la contrainte du nombre maximum de pas de temps imposé.

En revanche, une proportion non négligeable d'épisodes (~29%) ne sont pas résolus, mais ils sont compensés par les scores importants de ceux réussis.

Une vidéo de l'interaction de l'agent avec l'environnement est disponible sur [YouTube](https://youtu.be/G3rj4Yoc8bQ).

<img src="./images/youtube video.png" alt="Youtube video" width="400"/>

## Idées de travail futur

Quelques idées pour améliorer la méthode exploitant DQN :
- effectuer des recherches d'optimisation de valeurs des hyperparamètres liés à la structure du réseau de neurones (nombre de couches, taille des couches, etc.) et à l'algorithme DQN (taux d'apprentissage, taille du batch, le facteur de discount $\gamma$, le taux d'exploration $\epsilon$, le nombre de pas de temps par épisode, etc.)
- étudier l'importance de l'initialisation du réseau
- utilisation d'autres extensions de DQN, comme Double DQN, Dueling DQN, Prioritized Experience Replay, etc.
- étudier l'évolution d'obtention des récompenses négatives :
    - ajouter des pénalités pmlus importantes ?
    - comment augmenter des gains potentiels à moyen terme entrainer des actions produisant des récompenses négatives à court terme (en jouant sur $\gamma$ et la longueur d'une séquence)
- passer en mode d'apprentissage avec l'espace d'état pixel pour constater si l'agent peut être plus performant, et quel coût cela engendre par rapport à l'apprentissage avec l'espace d'état "discret".

Autres idées de constats pouvant fournir des pistes d'amélioration :
- étudier sur le taux d'échec des épisodes, alors que la moyenne des récompenses atteint l'objectif.
- combien d'épisodes d'entrainement est-il nécessaire pour augmenter le taux d'épisodes réussis en évaluation ?
- quelle moyenne de score maximale peut-on atteindre ?